<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=17392d5d2448261fce5a076107a122485748a4c48ce149b082d3427852b327bf
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

Matplotlib is building the font cache; this may take a moment.


In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-20 09:33:24
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 38.95 K (0.28%)
Current PnL: -21.04 L (-14.02%)
CY Booked + Current PnL: -9.14 L (-6.09%)
-------------------
Total profit:  1.98 L
Total loss:  -23.02 L
-------------------
Total Booked + Current PnL: 17.84 L (14.49%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.47%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.21 L (62.81%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-18.41,62.0,X-LC,3.95,217255.0,12755.0,10537.0,0.52,6.24,4.85,11.39,10.0,1.21,1.56,15.75,X40N,ATH,PHARMA
56,RELIANCE,1533.00,-9.16,72.0,X-LC,2.87,227011.0,16645.0,12145.0,2.71,7.91,5.35,13.68,37.0,1.37,1.63,25.74,XY25,NTT,REFINERIES
76,TTKPRESTIG,770.00,95.52,47.0,M-SC,1.70,84226.0,-16551.0,16643.0,0.12,-16.42,19.76,0.09,245.0,-0.99,0.60,10.66,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.02,56.0,X-MC,1.47,211995.0,20253.0,17511.0,-0.93,10.56,8.26,19.69,99.0,1.16,1.52,18.08,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.67,149177.0,13044.0,18080.0,0.74,9.58,12.12,22.86,91.0,0.72,1.07,23.40,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES
5,ASIANPAINT,3465.66,-12.54,65.0,X-LC,5.30,220686.0,-31082.0,84302.0,4.07,-12.35,38.20,21.13,27.0,-0.37,1.58,18.71,X40,ATH,PAINTS
56,RELIANCE,1533.00,-9.16,72.0,X-LC,2.87,227011.0,16645.0,12145.0,2.71,7.91,5.35,13.68,37.0,1.37,1.63,25.74,XY25,NTT,REFINERIES
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.68,157907.0,17067.0,27428.0,1.60,12.12,17.37,31.59,66.0,0.62,1.13,41.82,XY24,NTT,BANKS
45,KPIGREEN,731.05,2.88,61.0,H-SC,5.91,122422.0,-2875.0,61799.0,1.54,-2.29,50.48,47.03,141.0,-0.05,0.88,53.28,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,29.80,53.0,M-LC,10.11,211736.0,6660.0,105910.0,-2.70,3.25,50.02,54.89,52.0,0.06,1.52,28.13,X5K,ATH,METALS
36,INDIAMART,4810.62,-56.51,34.0,H-SC,7.71,121624.0,-1712.0,133336.0,-2.10,-1.39,109.63,106.72,119.0,-0.01,0.87,21.67,AR,ATH,MISC
70,TANLA,1963.11,-24.44,41.0,H-SC,18.88,202817.0,-67944.0,397886.0,-1.74,-25.09,196.18,121.86,133.0,-0.17,1.46,58.66,AR,ATH,IT
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229242.0,-18489.0,93554.0,-1.48,-7.46,40.81,30.30,92.0,-0.20,1.65,4.84,X40,ATH,ELECTRICAL
25,GREENPANEL,537.00,235.27,57.0,M-SC,4.25,149694.0,-33384.0,112360.0,-1.11,-18.23,75.06,43.14,230.0,-0.30,1.07,40.81,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,60.0,H-SC,12.68,131625.0,-10854.0,30379.0,0.06,-7.62,23.08,13.70,163.0,-0.36,0.94,55.35,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.24,57.0,H-SC,1.89,226215.0,-43452.0,79786.0,-0.75,-16.11,35.27,13.47,138.0,-0.54,1.62,15.02,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.52,47.0,M-SC,1.70,84226.0,-16551.0,16643.0,0.12,-16.42,19.76,0.09,245.0,-0.99,0.60,10.66,OX40N,NTT,DURABLES
17,CERA,9475.0,-24.58,37.0,H-SC,2.50,137701.0,-38202.0,80225.0,-0.25,-21.72,58.26,23.89,149.0,-0.48,0.99,18.92,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.55,45.0,H-MC,7.53,102144.0,-29691.0,71920.0,0.12,-22.52,70.41,32.03,98.0,-0.41,0.73,14.41,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,54.0,M-LC,6.03,208532.0,5362.0,36764.0,1.12,2.64,17.63,20.74,55.0,0.15,1.50,8.33,XY25,NTT,FINANCE
31,HINDZINC,730.22,29.80,53.0,M-LC,10.11,211736.0,6660.0,105910.0,-2.70,3.25,50.02,54.89,52.0,0.06,1.52,28.13,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.68,157907.0,17067.0,27428.0,1.60,12.12,17.37,31.59,66.0,0.62,1.13,41.82,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,57.0,H-MC,12.36,192096.0,12288.0,100120.0,1.45,6.83,52.12,62.52,88.0,0.12,1.38,43.25,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES
11,BANKINDIA,190.00,-25.47,57.0,H-MC,12.36,192096.0,12288.0,100120.0,1.45,6.83,52.12,62.52,88.0,0.12,1.38,43.25,XR,NTT,BANKS
31,HINDZINC,730.22,29.80,53.0,M-LC,10.11,211736.0,6660.0,105910.0,-2.70,3.25,50.02,54.89,52.0,0.06,1.52,28.13,X5K,ATH,METALS
36,INDIAMART,4810.62,-56.51,34.0,H-SC,7.71,121624.0,-1712.0,133336.0,-2.10,-1.39,109.63,106.72,119.0,-0.01,0.87,21.67,AR,ATH,MISC
85,WIPRO,420.00,-15.61,42.0,M-LC,6.03,148608.0,-2337.0,111798.0,-0.50,-1.55,75.23,72.51,53.0,-0.02,1.07,4.38,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,9.57,145672.0,-130851.0,240927.0,1.19,-47.32,165.39,39.81,54.0,-0.54,1.05,2.67,XY24,NTT,AUTO
51,QUESS,986.00,-48.76,32.0,X-SC,38.80,51136.0,-13870.0,164796.0,-0.26,-21.34,322.27,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229242.0,-18489.0,93554.0,-1.48,-7.46,40.81,30.30,92.0,-0.20,1.65,4.84,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,5.45,179449.0,-21057.0,118203.0,-0.01,-10.50,65.87,48.45,90.0,-0.18,1.29,3.48,X40N,ATH,FINANCE
55,RELAXO,1176.00,-39.88,42.0,X-SC,4.56,81949.0,-63211.0,139141.0,-0.54,-43.55,169.79,52.31,136.0,-0.45,0.59,9.69,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.40,47.0,H-LC,0.38,156995.0,-29100.0,76582.0,0.60,-15.64,48.78,25.51,15.0,-0.38,1.13,15.69,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.02,56.0,X-MC,1.47,211995.0,20253.0,17511.0,-0.93,10.56,8.26,19.69,99.0,1.16,1.52,18.08,XY25,NTT,AC
28,HAVELLS,2069.16,-7.45,39.0,X-MC,2.42,229242.0,-18489.0,93554.0,-1.48,-7.46,40.81,30.30,92.0,-0.20,1.65,4.84,X40,ATH,ELECTRICAL
41,ITC,452.00,-37.50,61.0,X-LC,2.59,202446.0,2308.0,18585.0,0.45,1.15,9.18,10.44,4.0,0.12,1.45,6.78,X40,NTT,FMCG
33,ICICIGI,2260.25,-13.38,66.0,X-MC,2.67,149177.0,13044.0,18080.0,0.74,9.58,12.12,22.86,91.0,0.72,1.07,23.40,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,32.0,X-SC,38.80,51136.0,-13870.0,164796.0,-0.26,-21.34,322.27,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,9.57,145672.0,-130851.0,240927.0,1.19,-47.32,165.39,39.81,54.0,-0.54,1.05,2.67,XY24,NTT,AUTO
2,ACC,3906.00,-38.07,45.0,X-MC,3.22,183710.0,-54241.0,206894.0,0.24,-22.80,112.62,64.15,174.0,-0.26,1.32,3.05,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,5.45,179449.0,-21057.0,118203.0,-0.01,-10.50,65.87,48.45,90.0,-0.18,1.29,3.48,X40N,ATH,FINANCE
73,TCS,4389.97,-29.65,47.0,X-LC,12.77,280393.0,-65563.0,132261.0,0.70,-18.95,47.17,19.28,1.0,-0.50,2.01,3.66,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.76,32.0,X-SC,38.80,51136.0,-13870.0,164796.0,-0.26,-21.34,322.27,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-39.88,42.0,X-SC,4.56,81949.0,-63211.0,139141.0,-0.54,-43.55,169.79,52.31,136.0,-0.45,0.59,9.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.01,43.0,X-SC,4.29,95749.0,-32921.0,78217.0,0.29,-25.59,81.69,35.20,219.0,-0.42,0.69,9.90,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.83,48.0,X-SC,0.82,107033.0,5029.0,38896.0,0.71,4.93,36.34,43.06,122.0,0.13,0.77,22.89,X40N,ATH,MISC
32,HONAUT,58357.33,-22.39,50.0,X-SC,2.54,109305.0,-18633.0,65769.0,0.34,-14.56,60.17,36.84,143.0,-0.28,0.78,11.90,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.65,47.0,X-LC,12.77,280393.0,-65563.0,132261.0,0.70,-18.95,47.17,19.28,1.0,-0.50,2.01,3.66,X40,ATH,IT
39,INFY,2275.00,-20.32,44.0,X-LC,7.63,310554.0,-2334.0,174034.0,1.17,-0.75,56.04,54.87,3.0,-0.01,2.23,5.85,X40,BTT,IT
41,ITC,452.00,-37.50,61.0,X-LC,2.59,202446.0,2308.0,18585.0,0.45,1.15,9.18,10.44,4.0,0.12,1.45,6.78,X40,NTT,FMCG
82,VBL,671.64,-18.24,54.0,X-LC,10.47,294469.0,-21373.0,134042.0,0.00,-6.77,45.52,35.67,5.0,-0.16,2.11,6.15,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,56.0,H-LC,6.88,252960.0,-8659.0,127871.0,1.31,-3.31,50.55,45.57,7.0,-0.07,1.82,7.64,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6800.00,59.0,L-SC,6.39,75685.0,-18125.0,94742.0,0.08,-19.32,125.18,81.67,269.0,-0.19,0.54,46.57,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,29.61,81024.0,-32525.0,72573.0,-0.40,-28.64,89.57,35.27,268.0,-0.45,0.58,98.31,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,42.0,H-SC,9.41,89745.0,-8235.0,29840.0,0.59,-8.40,33.25,22.05,152.0,-0.28,0.64,30.45,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-38.52,69.0,M-SC,4.16,103830.0,12332.0,66420.0,11.84,13.48,63.97,86.07,223.0,0.19,0.75,50.78,XR,NTT,DURABLES
45,KPIGREEN,731.05,2.88,61.0,H-SC,5.91,122422.0,-2875.0,61799.0,1.54,-2.29,50.48,47.03,141.0,-0.05,0.88,53.28,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-187.50,70.0,M-SC,19.63,150849.0,1629.0,142552.0,0.51,1.09,94.50,96.62,208.0,0.01,1.08,63.25,XY25,NTT,FINANCE
67,STARHEALTH,761.0,22.49,71.0,H-SC,9.85,273088.0,-8035.0,140886.0,-0.39,-2.86,51.59,47.26,144.0,-0.06,1.96,46.31,XY24,NTT,INSURANCE
4,ANGELONE,3033.0,15.82,70.0,X-SC,8.30,204967.0,13961.0,43740.0,0.12,7.31,21.34,30.21,157.0,0.32,1.47,29.72,X40N,NTT,FINANCE
66,SONACOMS,1006.0,-34.71,65.0,M-SC,11.41,81288.0,-19871.0,94766.0,0.50,-19.64,116.58,74.03,202.0,-0.21,0.58,14.62,AR,BTT,AUTO
49,NESTLEIND,1377.0,-1.48,75.0,X-LC,3.23,303264.0,37838.0,18954.0,0.50,14.26,6.25,21.40,11.0,2.00,2.18,21.85,XY25,NTT,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.33
2,50,75.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.23
LC    34.02
MC    21.78
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.57
X40      14.82
XY25     12.46
XR       10.90
X40N     10.26
AR        8.06
OX40N     7.61
X200      1.83
X5K       1.52
SR        1.12
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.32
X-LC    22.98
X-MC    16.09
M-SC    12.78
M-LC     5.22
H-LC     4.78
X-SC     4.67
H-MC     3.80
M-MC     1.59
L-SC     1.46
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.29,-3.04,37.01
IT,12.74,-18.58,82.59
FINANCE,12.19,-11.31,61.76
MISC,6.85,-20.45,87.19
BANKS,6.34,-12.78,72.11
PAINTS,5.71,-15.92,33.21
ELECTRICAL,5.38,-10.00,48.60
INSURANCE,4.00,0.73,36.38
AUTO,2.83,-46.00,115.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3313970.0,22
XR,1391396.0,14
AR,1208427.0,9
X40,771749.0,10
X40N,597456.0,8
OX40N,586052.0,10
XY25,447566.0,8
SR,265683.0,2
X5K,105910.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3472026.0,25
M-SC,1261343.0,15
X-MC,1085736.0,11
X-LC,1034966.0,13
X-SC,530559.0,6
H-MC,350698.0,3
M-LC,281900.0,4
H-LC,275286.0,3
L-SC,257348.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1191153.0      6
           AR         842166.0      5
           XR         836012.0      7
M-SC       XY24       692181.0      6
X-MC       XY24       584099.0      4
X-LC       X40        475873.0      5
           XY24       304910.0      2
X-SC       X40N       287546.0      4
H-SC       OX40N      275213.0      4
           SR         265683.0      2
X-MC       X40        217659.0      4
H-LC       AR         204453.0      2
X-LC       X40N       191707.0      3
H-MC       XY24       178658.0      1
M-MC       XY24       170745.0      1
L-SC       XR         167315.0      2
X-MC       XY25       165775.0      2
X-SC       XY24       164796.0      1
M-SC       AR         161808.0      2
           OX40N      148886.0      4
           XY25       142552.0      1
X-MC       X40N       118203.0      1
M-SC       XR         115916.0      2
M-LC       XR         111798.0      1
           X5K        105910.0      1
H-MC       XR         100120.0      1
L-SC       OX40N       90033.0      1
X-SC       X40         78217.0      1
H-MC       OX40N       71920.0      1
H-LC       X200        70833.0      1
X-LC       XY25        62476.0      3
H-SC       MH          61799.0      1
L-MC       XR          60235.0      1
L-LC       XY25        39999.0      1
M-LC       XY25        36764.0      1
           XY24        27428.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
